# Additional Field Configurations

We saw various model configurations that applied to all the fields in the model.

In [1]:
from pydantic import BaseModel, ConfigDict, Field, ValidationError

## Strict vs Lax Coercion

In [2]:
class Model(BaseModel):
    model_config = ConfigDict(strict=True)

    field_1: bool = False
    field_2: bool = False

This means that strict coercion mode will be enforced for **all** model fields.

In [3]:
try:
    Model(field_1=1.0, field_2=1.0)
except ValidationError as ex:
    print(ex)

2 validation errors for Model
field_1
  Input should be a valid boolean [type=bool_type, input_value=1.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/bool_type
field_2
  Input should be a valid boolean [type=bool_type, input_value=1.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/bool_type


In strict mode, Pydantic will not convert the floats `1.0` to `True` (or `0.0` to `False`).

Sometimes, we may want most of our fields to be in lax mode, and only some fields in strict mode. Or the other way around.

So, let's set our model to lax (the default), and only specify strict for some of the fields:

In [4]:
class Model(BaseModel):
    model_config = ConfigDict(
        strict=False, 
        validate_default=True,
    )

    field_1: bool = Field(strict=True, default=False)
    field_2: bool = False

In [5]:
Model(field_1=True, field_2=1.0)

Model(field_1=True, field_2=True)

But this will not work:

In [6]:
try:
    Model(field_1=1.0, field_2=0.0)
except ValidationError as ex:
    print(ex)

1 validation error for Model
field_1
  Input should be a valid boolean [type=bool_type, input_value=1.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/bool_type


Or, we could do it the other way around, set our entire model to be strict, and some fields to be lax:

In [7]:
class Model(BaseModel):
    model_config = ConfigDict(
        strict=True, 
        validate_default=True,
    )

    field_1: bool = Field(strict=False, default=False)
    field_2: bool = False

In [8]:
Model(field_1=1.0, field_2=True)

Model(field_1=True, field_2=True)

But this will not work:

In [9]:
try:
    Model(field_1=1.0, field_2=1.0)
except ValidationError as ex:
    print(ex)

1 validation error for Model
field_2
  Input should be a valid boolean [type=bool_type, input_value=1.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/bool_type


## Validate Defaults

The same thing can be done with `validate_default` - we can specify it at the model level, or we can override the model level configuration at the field level.

In [10]:
class Model(BaseModel):
    model_config = ConfigDict(
        strict=False, 
        validate_default=True,
    )

    field_1: bool = Field(strict=True, default=1.0, validate_default=False)
    field_2: bool = False

And now, even though `field_1` is set to strict, because we skip default validation explicitly for `field_1` we end up with inconsistent data:

In [11]:
Model()

Model(field_1=1.0, field_2=False)

## Frozen Fields

We saw how we could freeze our entire model:

In [12]:
class Model(BaseModel):
    model_config = ConfigDict(frozen=True)

    field_1: int
    field_2: int

In [13]:
m = Model(field_1=1, field_2=2)

In [14]:
try:
    m.field_1=10
except ValidationError as ex:
    print(ex)

1 validation error for Model
field_1
  Instance is frozen [type=frozen_instance, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_instance


We could alternatively, choose to only freeze certain fields.

Note that you cannot set the model as a whole to be frozen and then set individual fields to not be frozen - if you set the model as frozen, then the entire model is frozen.

In [15]:
class Model(BaseModel):
    field_1: int = Field(frozen=True)
    field_2: int

In [16]:
m = Model(field_1=1, field_2=2)
m

Model(field_1=1, field_2=2)

In [17]:
m.field_2 = 20
m

Model(field_1=1, field_2=20)

In [18]:
try:
    m.field_1 = 10
except ValidationError as ex:
    print(ex)

1 validation error for Model
field_1
  Field is frozen [type=frozen_field, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_field


## Excluding Fields from Serialization

We saw earlier how we could provide arguments to `model_dump()` and `model_dump_json()` to specify fields to include or exclude from being serialized.

In [19]:
class Model(BaseModel):
    field_1: int = 1
    field_2: int = 2
    field_3: int = 3

In [20]:
m = Model()

In [21]:
m.model_dump(exclude=['field_1'])

{'field_2': 2, 'field_3': 3}

Sometimes, we want to always exclude a field from serialization - maybe some secret value that we need in our model, but that we do not want to accidentally serialize in the output of a REST endpoint for example.

Instead on relying to remembering to exclude that field from serialization every time we call one of the dump methods, we can set it at the field level:

In [22]:
class Model(BaseModel):
    key: str = Field(default='python', exclude=True)
    field_1: int = 1
    field_2: int = 2
    field_3: int = 3

In [23]:
m = Model()
m

Model(key='python', field_1=1, field_2=2, field_3=3)

In [24]:
m.model_dump()

{'field_1': 1, 'field_2': 2, 'field_3': 3}

Note that once you have excluded a field using a field level definition, you cannot "un-exclude" it when serializing:

In [25]:
m.model_dump(include=['key', 'field_1', 'field_2', 'field_3'])

{'field_1': 1, 'field_2': 2, 'field_3': 3}

As you can see, `key` was not serialized.